In [1]:
##### STARTING PYVISA AND THE RESOURCES
import pyvisa
rm = pyvisa.ResourceManager()
print(rm.list_resources())

('ASRL1::INSTR', 'ASRL2::INSTR', 'GPIB0::11::INSTR')


In [2]:
import time
import numpy as np
import serial

In [3]:
##### LECROY CHANNELS: EA,EB,MC,MD,FE,FF,C1,C2
##### TRIGGERS: EX,EX1O

In [4]:
##### INSTRUMENT OPENS AND IDENTIFIES
lecroy_9410 = rm.open_resource('GPIB0::11::INSTR')
print(lecroy_9410.query('*IDN?'))

*IDN LECROY,9410_,94102018,05.0



In [5]:
##### LONG AND SHORT FORM COMMANDS
print(lecroy_9410.query('ALL_STATUS?'))
print(lecroy_9410.query('ALST?'))

ALST STB,000000,ESR,000000,INR,000001,DDR,000000,EXR,000000,CMR,000000,URR,000000

ALST STB,000000,ESR,000000,INR,000000,DDR,000000,EXR,000000,CMR,000000,URR,000000



In [22]:
##### ADDRESS OF THE INSTRUMENT
print(lecroy_9410)

GPIBInstrument at GPIB0::11::INSTR


In [ ]:
##### INSTRUMENT SESSION
lecroy_9410.session

In [8]:
##### CLOSE SESSION
lecroy_9410.close()

In [42]:
##### EXAMPLE SET OFFSETS FOR CHANNELS
lecroy_9410.write('C1:OFST 0 MV')
lecroy_9410.write('C2:OFST 1 MV')

14

In [55]:
##### EXAMPLE SET VOLTAGE AND TIME DIVISIONS
lecroy_9410.write('C1:TIME_DIV 20 MS')
lecroy_9410.write('C2:TIME_DIV 20 MS')
lecroy_9410.write('C1:VOLT_DIV 1 MV')
lecroy_9410.write('C2:VOLT_DIV 1 MV')

18

In [35]:
##### PAVA GETS PARAMETER VALUES
print(lecroy_9410.query('C1:PAVA?'))

C1:PAVA FRST,-0.05E-6 S,OK,LAST,0.45E-6 S,OK,PNTS,50,OK,MIN,-1.6E-3 V,OK,MAX,0E-3 V,OK,MEAN,-0.16E-3 V,OK,SDEV,0.47E-3 V,OK,RMS,0.49E-3 V,OK,BASE,0E-3 V,OK,TOP,-1.6E-3 V,OK,AMPL,-1.6E-3 V,OK,PKPK,1.6E-3 V,OK,FREQ,0,IV,PER,0,IV,WID,0,IV,RISE,0,IV,FALL,0,IV,DLY,0,IV,CYCL,0,IV,DUTY,0,IV,AREA,-78.1E-12 V.S,OK,OVSP,0 PCT,OK,OVSN,0 PCT,OK,ALLI,0,IV,ANYO,0,IV



In [79]:
##### SHOW TRACES
lecroy_9410.write('C1:TRACE OFF')
lecroy_9410.write('FE:TRACE ON')

13

In [80]:
##### SHOW PARAMETERS
lecroy_9410.write('FE:CRMS PARAM')

15

In [93]:
##### PAVA FOR THE FUNCTION
print(lecroy_9410.query('FE:PAVA? MIN,MAX'))

FE:PAVA MIN,-81.4E-6 V,OK,MAX,-42.1E-6 V,OK



In [7]:
##### FUNCTION DEFINITION
lecroy_9410.query('FE:DEF?')

'FE:DEF EQN,"AVGS(C1)",MAXPTS,1000,SWEEPS,250\n'

In [28]:
##### RESTART THE FUNCTION
lecroy_9410.write('FE:FRST')

9

In [6]:
##### DEFINE FUNCTION
lecroy_9410.write('FE:DEF EQN,\'AVGS(C1)\',MAXPTS,1000,SWEEPS,250')

46

In [105]:
##### SAVE QUERY TO STRING
a=lecroy_9410.query('FE:PAVA? MIN')
print(a)

FE:PAVA MIN,-81.4E-6 V,OK



In [108]:
##### SPLIT STRING AND SAVE NUMERICAL DATA
a=lecroy_9410.query('FE:PAVA? MIN')
a=a.split()
stream=a[1].split(',')
scopemin=float(stream[1])
a=lecroy_9410.query('FE:PAVA? MAX')
a=a.split()
stream=a[1].split(',')
scopemax=float(stream[1])
print("Min is: %.8f, Max is: %.8f" %(scopemin,scopemax))

Min is: -0.00008140, Max is: -0.00004210


In [134]:
##### TIMED TEST
wavels = np.arange(614.07,614.073,.001)
for i in wavels:
    lecroy_9410.write('FE:FRST')
    time.sleep(60)
    a=lecroy_9410.query('FE:PAVA? MIN')
    a=a.split()
    stream=a[1].split(',')
    scopemin=float(stream[1])
    a=lecroy_9410.query('FE:PAVA? MAX')
    a=a.split()
    stream=a[1].split(',')
    scopemax=float(stream[1])
    print("For wavelength %f, Min is: %.8f, Max is: %.8f" %(i,scopemin,scopemax))

For wavelength 614.070000, Min is: -0.00007400, Max is: -0.00003900
For wavelength 614.071000, Min is: -0.00007500, Max is: -0.00003800
For wavelength 614.072000, Min is: -0.00007600, Max is: -0.00004100


In [124]:
##### TRIGGERS
lecroy_9410.write('TRMD AUTO')

11

In [129]:
lecroy_9410.write('TRSE EV,SR,EX')

15

In [127]:
lecroy_9410.write('C1:TRCP DC')

12

In [130]:
lecroy_9410.query('TRSE?')

'TRSE STD,SR,C1\n'

In [21]:
##### FULL SERIAL + GPIB TEST
ser = serial.Serial('COM1',9600,stopbits=serial.STOPBITS_TWO)
wavels = np.arange(614.07,614.074,.001)
for i in wavels:
    commnd = 'START=%.3f\r' %i
    ser.write(bytes(commnd,'utf-8'))
    time.sleep(1)
    ser.write(bytes('GOSTART\r','utf-8'))
    time.sleep(5)
    lecroy_9410.write('FE:FRST')
    time.sleep(30)
    a=lecroy_9410.query('FE:PAVA? MIN')
    a=a.split()
    stream=a[1].split(',')
    scopemin=float(stream[1])
    a=lecroy_9410.query('FE:PAVA? MAX')
    a=a.split()
    stream=a[1].split(',')
    scopemax=float(stream[1])
    print("For wavelength %f, Min is: %.8f, Max is: %.8f" %(i,scopemin,scopemax))

For wavelength 614.070000, Min is: 0.00023000, Max is: 0.00095000
For wavelength 614.071000, Min is: 0.00031000, Max is: 0.00098000
For wavelength 614.072000, Min is: 0.00020000, Max is: 0.00087000
For wavelength 614.073000, Min is: 0.00017000, Max is: 0.00082000


In [10]:
ser.close()
lecroy_9410.close()

In [11]:
##### FULL RUN
import pyvisa
import time
import datetime as dt
import numpy as np
import serial
rm = pyvisa.ResourceManager()
print(rm.list_resources())
lecroy_9410 = rm.open_resource('GPIB0::11::INSTR')
print(lecroy_9410.query('*IDN?'))
ser = serial.Serial('COM2',9600,stopbits=serial.STOPBITS_TWO)
minV = []
maxV = []
wavels = np.arange(614.06,614.211,.001)
for i in wavels:
    commnd = 'START=%.3f\r' %i
    ser.write(bytes(commnd,'utf-8'))
    time.sleep(1)
    ser.write(bytes('GOSTART\r','utf-8'))
    time.sleep(5)
    lecroy_9410.write('FE:FRST')
    time.sleep(30)
    a=lecroy_9410.query('FE:PAVA? MIN')
    a=a.split()
    stream=a[1].split(',')
    scopemin=float(stream[1])
    a=lecroy_9410.query('FE:PAVA? MAX')
    a=a.split()
    stream=a[1].split(',')
    scopemax=float(stream[1])
    minV.append(scopemin)
    maxV.append(scopemax)
    print("For wavelength %f, Min is: %.8f, Max is: %.8f" %(i,scopemin,scopemax))
now=dt.datetime.now()
dtime=now.strftime('_%y%m%d_%H%M')
maxV=np.asarray(maxV)
minV=np.asarray(minV)
np.save('minV'+dtime,minV)
np.save('maxV'+dtime,maxV)
np.save('waves'+dtime,wavels)
ser.close()
lecroy_9410.close()

('ASRL1::INSTR', 'ASRL2::INSTR', 'GPIB0::11::INSTR')
*IDN LECROY,9410_,94102018,05.0

For wavelength 614.060000, Min is: -0.14900000, Max is: 0.09100000
For wavelength 614.061000, Min is: -0.14500000, Max is: 0.09100000
For wavelength 614.062000, Min is: -0.14200000, Max is: 0.09000000
For wavelength 614.063000, Min is: -0.14600000, Max is: 0.09200000
For wavelength 614.064000, Min is: -0.14800000, Max is: 0.09400000
For wavelength 614.065000, Min is: -0.14800000, Max is: 0.09400000
For wavelength 614.066000, Min is: -0.15200000, Max is: 0.09600000
For wavelength 614.067000, Min is: -0.15400000, Max is: 0.10000000
For wavelength 614.068000, Min is: -0.15500000, Max is: 0.10100000
For wavelength 614.069000, Min is: -0.16200000, Max is: 0.10700000
For wavelength 614.070000, Min is: -0.16400000, Max is: 0.11200000
For wavelength 614.071000, Min is: -0.16800000, Max is: 0.11500000
For wavelength 614.072000, Min is: -0.17500000, Max is: 0.12100000
For wavelength 614.073000, Min is: -0.18200

For wavelength 614.181000, Min is: -0.26200000, Max is: 0.19500000
For wavelength 614.182000, Min is: -0.25600000, Max is: 0.19400000
For wavelength 614.183000, Min is: -0.24800000, Max is: 0.19000000
For wavelength 614.184000, Min is: -0.23200000, Max is: 0.18000000
For wavelength 614.185000, Min is: -0.22900000, Max is: 0.17900000
For wavelength 614.186000, Min is: -0.22400000, Max is: 0.17500000
For wavelength 614.187000, Min is: -0.21600000, Max is: 0.16700000
For wavelength 614.188000, Min is: -0.21000000, Max is: 0.16400000
For wavelength 614.189000, Min is: -0.20300000, Max is: 0.16100000
For wavelength 614.190000, Min is: -0.19900000, Max is: 0.15900000
For wavelength 614.191000, Min is: -0.19100000, Max is: 0.15600000
For wavelength 614.192000, Min is: -0.18700000, Max is: 0.15500000
For wavelength 614.193000, Min is: -0.18400000, Max is: 0.15400000
For wavelength 614.194000, Min is: -0.17900000, Max is: 0.15200000
For wavelength 614.195000, Min is: -0.17500000, Max is: 0.1500

In [35]:
wavels

array([614.06 , 614.061, 614.062, 614.063, 614.064, 614.065, 614.066,
       614.067, 614.068, 614.069, 614.07 , 614.071, 614.072, 614.073,
       614.074, 614.075, 614.076, 614.077, 614.078, 614.079, 614.08 ,
       614.081, 614.082, 614.083, 614.084, 614.085, 614.086, 614.087,
       614.088, 614.089, 614.09 , 614.091, 614.092, 614.093, 614.094,
       614.095, 614.096, 614.097, 614.098, 614.099, 614.1  , 614.101,
       614.102, 614.103, 614.104, 614.105, 614.106, 614.107, 614.108,
       614.109, 614.11 , 614.111, 614.112, 614.113, 614.114, 614.115,
       614.116, 614.117, 614.118, 614.119, 614.12 , 614.121, 614.122,
       614.123, 614.124, 614.125, 614.126, 614.127, 614.128, 614.129,
       614.13 , 614.131, 614.132, 614.133, 614.134, 614.135, 614.136,
       614.137, 614.138, 614.139, 614.14 , 614.141, 614.142, 614.143,
       614.144, 614.145, 614.146, 614.147, 614.148, 614.149, 614.15 ,
       614.151, 614.152, 614.153, 614.154, 614.155, 614.156, 614.157,
       614.158, 614.